In [1]:
import urllib
import urllib.request
import datetime
import pandas as pd
import numpy as np
from pandas import read_csv  
import random as rd # generating random numbers
#from datetime import datetime
import os
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [125]:
df.describe()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
count,10267862,10267862,10267862,10267862,10267862,10267862,10267862,10267862,10267862,10267862,10267862
unique,732,468,220,377,129,7,372,85384,3,6860803,6738931
top,PTH05,R549,00-00-00,34 ST-PENN STA,1,IRT,02/19/2016,04:00:00,REGULAR,0,0
freq,54629,131650,1028471,223582,1157633,3790711,34773,794799,10225225,134111,258467


# Data Description

The data file consists of 10267862 rows and 11 columns namely:
1. C/A      = Control Area (A002)
2. UNIT     = Remote Unit for a station (R051)
3. SCP      = Subunit Channel Position represents an specific address for a device (02-00-00)
4. STATION  = Represents the station name the device is located at
5. LINENAME = Represents all train lines that can be boarded at this station. LINENAME 456NQR repersents train server for 4, 5, 6, N, Q, and R trains.
6. DIVISION = Represents the Line originally the station belonged to
    - IRT = Interborough Rapid Transit;
    - BMT = Brooklyn-Manhattan Transit;
    - IND = Independent Subway;
    - PTH = Port Authority Trans-Hudson (PATH);
    - RIT = Roosevelt Island Tram;
    - SRT = Staten Island Rapid Transit;
7. DATE     = Represents the date (MM-DD-YY)
8. TIME     = Represents the time (hh:mm:ss) for a scheduled audit event
9. DESc     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
10. ENTRIES  = The cumulative entry register value for a device
11. EXIST    = The cumulative exit register value for a device

In [126]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,03:00:00,REGULAR,5980509,2025806
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,07:00:00,REGULAR,5980531,2025818
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,11:00:00,REGULAR,5980613,2025912
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,15:00:00,REGULAR,5980899,2025965
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,19:00:00,REGULAR,5981382,2026001


In [127]:
df.tail()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
10267857,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/01/2016,04:00:00,REGULAR,5554,230
10267858,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/01/2016,08:00:00,REGULAR,5554,230
10267859,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/01/2016,12:00:00,REGULAR,5554,230
10267860,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/01/2016,16:00:00,REGULAR,5554,230
10267861,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,01/01/2016,20:00:00,REGULAR,5554,230


# Data Cleaning 

In [128]:
df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

#### Remove white space from header

In [129]:
df.columns = df.columns.str.strip()
df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

#### Drop duplicate value

In [130]:
df.duplicated().value_counts()

False    10267811
True           51
dtype: int64

In [131]:
df = df.drop_duplicates()

#### Drop NA values, if any

In [132]:
df = df.dropna()

#### Check unique values 

In [133]:
df['C/A'].unique()
df = df.drop(df[df['C/A'] == 'C/A'].index)
# df['UNIT'].unique()
# df = df.drop(df[df['UNIT'] == 'UNIT'].index)
# df['SCP'].unique()
# df = df.drop(df[df['SCP'] == 'SCP'].index)
# df = df.drop(df[df['SCP'] == ''].index)
# df['LINENAME'].unique()
# df = df.drop(df[df['LINENAME'] == 'LINENAME'].index)
# df['DIVISION'].unique()
# df = df.drop(df[df['DIVISION'] == 'DIVISION'].index)
# df['DATE'].unique()
# df = df.drop(df[df['DATE'] == 'DATE'].index)
# df['DESC'].unique()
# df = df.drop(df[df['DESC'] == 'DESC'].index)

#### Renaming station names 

In [134]:
df['STATION'] = df['STATION'].replace(['AVENUE', 'AVEE', 'AV'], 'AVE')

In [135]:
df['STATION'] = df['STATION'].replace(['PKWY'], 'PARKWAY')

In [136]:
df.loc[df['STATION'] == 'TIMES SQ-42 ST', 'STATION'] = '42 ST-TIMES SQ'
df.loc[df['STATION'] == '137 ST CITY COL', 'STATION'] = '137 ST-CITY COL'
df.loc[df['STATION'] == '182-183 STS', 'STATION'] = '182-183 ST'
df.loc[df['STATION'] == '33 ST-RAWSON ST', 'STATION'] = '33 ST/RAWSON ST'
df.loc[df['STATION'] == '40 ST LOWERY ST', 'STATION'] = '40 ST-LOWERY ST'
df.loc[df['STATION'] == '5 AVE/59 ST', 'STATION'] = '5 AVE-59 ST'
df.loc[df['STATION'] == '59 ST COLUMBUS', 'STATION'] = '59 ST-COLUMBUS'
df.loc[df['STATION'] == 'ASTOR PL', 'STATION'] = 'ASTOR PLACE'
df.loc[df['STATION'] == 'BEDFORD PK BLVD', 'STATION'] = 'BEDFORD PARK BL'
df.loc[df['STATION'] == 'BEVERLY RD', 'STATION'] = 'BEVERLY ROAD'
df.loc[df['STATION'] == 'CORTELYOU RD', 'STATION'] = 'CORTELYOU ROAD'
df.loc[df['STATION'] == 'FRESH POND RD', 'STATION'] = 'FRESH POND ROAD'
df.loc[df['STATION'] == 'HOWARD BCH JFK', 'STATION'] = 'HOWARD BCH-JFK'
df.loc[df['STATION'] == 'NECK RD', 'STATION'] = 'NECK ROAD'
df.loc[df['STATION'] == 'VERNON-JACKSON', 'STATION'] = 'VERNON/JACKSON'
df.loc[df['STATION'] == 'WAKEFIELD/241', 'STATION'] = 'WAKEFIELD-241'
df.loc[df['STATION'] == 'GRD CNTRL-42 ST', 'STATION'] = '42 ST-GRD CNTRL'

#### Drop stations names that do not exists

In [137]:
df.loc[df['STATION'] == '161/YANKEE STAD', 'STATION'] = 0
df.loc[df['STATION'] == '168 ST', 'STATION'] = 0
df.loc[df['STATION'] == '138/GRAND CONC', 'STATION'] = 0
df.loc[df['STATION'] == '3 AVE-149 ST', 'STATION'] = 0
df.loc[df['STATION'] == '30 AVE', 'STATION'] = 0
df.loc[df['STATION'] == '34 ST-HUDSON YD', 'STATION'] = 0
df.loc[df['STATION'] == '36 AVE', 'STATION'] = 0
df.loc[df['STATION'] == '4 AVE-9 ST', 'STATION'] = 0
df.loc[df['STATION'] == '42 ST-PORT AUTH', 'STATION'] = 0
df.loc[df['STATION'] == '80 ST', 'STATION'] = 0
df.loc[df['STATION'] == '88 ST', 'STATION'] = 0
df.loc[df['STATION'] == 'BROADWAY JCT', 'STATION'] = 0
df.loc[df['STATION'] == "B'WAY-LAFAYETTE", 'STATION'] = 0
df.loc[df['STATION'] == 'CANARSIE-ROCKAW', 'STATION'] = 0
df.loc[df['STATION'] == 'CATHEDRAL PARKWAY', 'STATION'] = 0
df.loc[df['STATION'] == 'EASTCHSTER/DYRE', 'STATION'] = 0
df.loc[df['STATION'] == 'FLATBUSH AVE-B.C', 'STATION'] = 0
df.loc[df['STATION'] == 'HARLEM 148 ST', 'STATION'] = 0
df.loc[df['STATION'] == 'KEW GARDENS', 'STATION'] = 0
df.loc[df['STATION'] == 'LEXINGTON AVE/63', 'STATION'] = 0
df.loc[df['STATION'] == 'MORISN AVE/SNDVW', 'STATION'] = 0
df.loc[df['STATION'] == 'NEWKIRK PLAZA', 'STATION'] = 0
df.loc[df['STATION'] == 'OZONE PK LEFFRT', 'STATION'] = 0
df.loc[df['STATION'] == 'PARKCHESTER', 'STATION'] = 0
df.loc[df['STATION'] == 'PATH NEW WTC', 'STATION'] = 0
df.loc[df['STATION'] == 'SUTPHIN-ARCHER', 'STATION'] = 0
df.loc[df['STATION'] == 'SUTTER AVE-RUTLD', 'STATION'] = 0
df.loc[df['STATION'] == 'V.CORTLANDT PK', 'STATION'] = 0

In [138]:
df = df.drop(df[df['STATION'] == 0].index)

#### Format date 

In [139]:
df['DATETIME'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'], format = "%m/%d/%Y %H:%M:%S", errors = 'coerce')

In [140]:
df['YEAR'] = df['DATETIME'].dt.year.astype(int)
df['MONTH'] = df['DATETIME'].dt.month.astype(int)
df['DAY'] = df['DATETIME'].dt.day.astype(int)
df['WEEKDAY'] = df['DATETIME'].dt.weekday_name
df['HOUR'] = df['DATETIME'].dt.hour.astype(int)
df['TIME'] = df['DATETIME'].dt.time

In [141]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,YEAR,MONTH,DAY,WEEKDAY,HOUR
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,03:00:00,REGULAR,5980509,2025806,2016-12-24 03:00:00,2016,12,24,Saturday,3
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,07:00:00,REGULAR,5980531,2025818,2016-12-24 07:00:00,2016,12,24,Saturday,7
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,11:00:00,REGULAR,5980613,2025912,2016-12-24 11:00:00,2016,12,24,Saturday,11
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,15:00:00,REGULAR,5980899,2025965,2016-12-24 15:00:00,2016,12,24,Saturday,15
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/24/2016,19:00:00,REGULAR,5981382,2026001,2016-12-24 19:00:00,2016,12,24,Saturday,19


In [142]:
df['MONTH'].unique()

array([12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1], dtype=int64)

In [143]:
df['WEEKDAY'].unique()

array(['Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',
       'Friday'], dtype=object)

In [148]:
df['HOUR'].unique()

array([ 3,  7, 11, 15, 19, 23,  1,  5,  9, 13, 17, 21,  8,  0,  4, 12, 16,
       20,  2,  6, 10, 14, 18, 22], dtype=int64)

#### Check unique year

In [149]:
df['YEAR'].unique()

array([2016, 2015], dtype=int64)

#### Drop 2015 Year  

In [150]:
df = df.drop(df[df['YEAR'] == 2015].index)

In [151]:
df.count()

C/A         9557107
UNIT        9557107
SCP         9557107
STATION     9557107
LINENAME    9557107
DIVISION    9557107
DATE        9557107
TIME        9557107
DESC        9557107
ENTRIES     9557107
EXITS       9557107
DATETIME    9557107
YEAR        9557107
MONTH       9557107
DAY         9557107
WEEKDAY     9557107
HOUR        9557107
dtype: int64

After data cleaning and preprocessing, the total number of records are: 9557107 rows x 17 columns.